# CS670 Group Project Phase 3
Team members:
Alexander Burke Alexander.Burke001@umb.edu
Divya Bajaj : Divya.Bajaj001@umb.edu
Yu-Ju Chien : Yuju.Chien001@umb.edu
Oleksandr Pimenov: pialgi@live.com

In [12]:
import numpy as np
from sklearn import preprocessing

# Task 1: Split the experimental samples and normalize the samples
data0 = np.load('data_selected_1980_2010.npy').astype(np.float)
data1 = np.load('target_1980_2010.npy').astype(np.float)
# Concatenate data_selected(features) with target
data = np.concatenate((data0, data1), axis = 1)

normalized_x = preprocessing.normalize(data0)
print(data0)
print(" ")
print(normalized_x)

# Split the data to training set(first 9497 rows) and testing set(last 1803 rows)
# split = 9497
# train, test = data[:split, :], data[split:, :]
# print("Training Set: ")
# print(train)
# print(train.shape)
# print("Testing Set: ")
# print(test)
# print(test.shape)

####################################################################
split = 9497
x_train, x_test, y_train, y_test = data0[:split, :], data0[split:, :], data1[:split, 1:], data1[split:, 1:]
#np.array(['1','2','3']).astype(np.float)
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

[[  12.66      15.85     279.88     290.75   ]
 [  13.5      -13.42     279.48     290.91   ]
 [  11.28       8.75     279.83     291.23   ]
 ..., 
 [   0.70001   -1.88     284.47     288.55   ]
 [   5.12      -2.23     282.5      289.02   ]
 [   7.7        3.42     282.4      289.55   ]]
 
[[ 0.0313305   0.039225    0.69263676  0.71953744]
 [ 0.03342772 -0.03322963  0.69202804  0.72033018]
 [ 0.0279116   0.02165128  0.69242044  0.72062897]
 ..., 
 [ 0.00172756 -0.00463967  0.70204566  0.71211472]
 [ 0.01266729 -0.0055172   0.69892778  0.71505878]
 [ 0.01903354  0.00845386  0.69806118  0.71573518]]
((9497L, 4L), (1803L, 4L))
((9497L, 1L), (1803L, 1L))


In [17]:
# Naive Bayes Classifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

gnb = GaussianNB()
y_pred = gnb.fit(x_train, y_train).predict(x_test)
print(y_pred)
print "Accuracy: %f" % metrics.accuracy_score(y_test, y_pred)
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

[ 0.  0.  0. ...,  0.  0.  0.]
Accuracy: 0.933999
[[1684    0]
 [ 119    0]]
             precision    recall  f1-score   support

        0.0       0.93      1.00      0.97      1684
        1.0       0.00      0.00      0.00       119

avg / total       0.87      0.93      0.90      1803



In [ ]:
# ROC curve and AUC


In [18]:
from sklearn.metrics import roc_auc_score

# ROC_AUC_Score: Computing the area under the ROC-curve
ras = roc_auc_score(y_test, y_pred)
print "roc_auc_score = %f" % ras

roc_auc_score = 0.500000
